# Modelling tides

**This guide demonstrates how to use the [`model_tides`](../../api/#eo_tides.model.model_tides) function from the [`eo_tides.model`](../../api/#eo_tides.model) module to model tide heights at multiple coordinates or time steps, using one or more ocean tide models.**

The `model_tides` function supports tide modelling based on a wide range of ocean tide models using a single line of code, parallelising this modelling where possible and returning data in a standardised `pandas.Dataframe` format.
The `model_tides` function can be used independently of Earth observation (EO) data, e.g. for any application where you need to generate a time series of tide heights.
However, it also underpins the more complex EO-related functions demonstrated in [Combining tides with satellite data](../Satellite_data).

<div class="admonition tip">
    <p class="admonition-title">Tip</p>
    <p>
        The <code>model_tides</code> function is based on the <a href="https://pytmd.readthedocs.io/en/latest/api_reference/compute.html#pyTMD.compute.tide_elevations"><code>pyTMD.compute.tide_elevations</code></a> function from the <code>pyTMD</code> tide modelling package that underpins <code>eo-tides</code>, with modifications to support parallel processing and integration with <code>pandas</code> and <code>xarray</code> workflows. We highly recommend exploring the <a href="https://pytmd.readthedocs.io/en/latest/getting_started/Overview.html">more advanced tide modelling functionality available in <code>pyTMD</code></a> for more custom tide modelling applications.
    </p>
</div>

## Getting started
As a first step, we need to tell `eo-tides` the location of our tide model directory (if you haven't set this up, [refer to the setup instructions here](../../setup)).

We will pass this path to `eo-tides` functions using the `directory` parameter.

<!-- For local installation:
pip install -e ../../. -->

In [ ]:
directory = "../../tests/data/tide_models/"

We can use the [`eo_tides.model.list_models`](../../api/#eo_tides.model.list_models) function to verify that we have some tide model data available in this directory:

In [ ]:
from eo_tides.model import list_models

list_models(directory=directory, show_supported=False);

<div class="admonition important">
    <p class="admonition-title">Important</p>
    <p>
        The directory above contains demo data only.
        Update the <code>directory</code> path to point to the location of your own tide model directory.
    </p>
</div>

## Using model_tides

In the example below, we use the [`model_tides`](../../api/#eo_tides.model.model_tides) function to model hourly tides for the city of **Broome, Western Australia** across **January 2018**:

In [ ]:
from eo_tides.model import model_tides
import pandas as pd

tide_df = model_tides(
    x=122.2186,
    y=-18.0008,
    time=pd.date_range(start="2018-01-01", end="2018-01-31", freq="1h"),
    directory=directory,
)

# Print outputs
tide_df.head()

The resulting `pandas.DataFrame` contains:

* `time`, `x`, `y`: Our original input timesteps and coordinates
* `tide_model`: a column listing the tide model used
* `tide_height`: modelled tide heights representing tide height in metres relative to Mean Sea Level

We can plot our modelled outputs to view how tides changed across the month. 
Looking at the y-axis, we can see that tides at this macrotidal region ranged from -4 metres up to a maximum of +4 metres relative to Mean Sea Level:

In [ ]:
tide_df.droplevel(["x", "y"]).tide_height.plot();

### Multiple models

By default, `model_tides` will model tides using the [`EOT20` tide model](https://www.seanoe.org/data/00683/79489/) – a leading open-source global ocean model with a permissive CC BY 4.0 licence:

> Hart-Davis Michael, Piccioni Gaia, Dettmering Denise, Schwatke Christian, Passaro Marcello, Seitz Florian (2021). EOT20 - A global Empirical Ocean Tide model from multi-mission satellite altimetry. SEANOE. https://doi.org/10.17882/79489

However, we can easily model tides using multiple models by passing a list of models to the `model` parameter.
`eo-tides` will process these in parallel where possible, and return the data into a single `pandas.DataFrame`.
For example, we can model tides using the `EOT20`, `GOT5.5` and `HAMTIDE11` models:

<div class="admonition note">
    <p class="admonition-title">Note</p>
    <p>
        Here we also set <code>output_format="wide"</code>, which will place data from each model into a new column.
        This can make it easier to plot our data. For more details, <a href="#wide-and-long-output-formats">see below</a>.
    </p>
</div>

In [ ]:
tide_df_multiple = model_tides(
    x=122.2186,
    y=-18.0008,
    model=["EOT20", "HAMTIDE11", "GOT5.5"],
    time=pd.date_range(start="2018-01-01", end="2018-01-31", freq="1h"),
    output_format="wide",
    directory=directory,
)

# Print outputs
tide_df_multiple.head()

Plot our outputs to see all our models on a graph:

In [ ]:
# Print outputs
tide_df_multiple.droplevel(["x", "y"]).plot(legend=True)

### "One-to-many" and "one-to-one" modes

By default, the `model_tides` function operates in **"one-to-many"** mode, which will model tides at every requested location, for every requested timestep.
This is particularly useful for satellite Earth observation applications where we may want to model tides for a large set of satellite pixels, for every satellite acquisition through time.

For example, if we provide two locations and two timesteps, the function will return four modelled tides:
```
2 locations * 2 timesteps = 4 modelled tides
```

In [ ]:
model_tides(
    x=[122.21, 122.22],
    y=[-18.20, -18.21],
    time=pd.date_range(start="2018-01-01", end="2018-01-31", periods=2),
    mode="one-to-many",
    directory=directory,
)

However, another common use case is having a list of locations you want to use to model tides for, each with a single timestep.
Using **"one-to-one"** mode, we can model tides for each pair of locations and times:
```
2 timesteps at 2 locations = 2 modelled tides
```

For example, you may have a `pandas.DataFrame` containing `x`, `y` and `time` values:

In [ ]:
df = pd.DataFrame(
    {
        "time": pd.date_range(start="2018-01-01", end="2018-01-31", periods=2),
        "x": [122.21, 122.22],
        "y": [-18.20, -18.21],
    }
)
df

We can pass these values to `model_tides` directly, and run the function in "one-to-one" mode to return a tide height for each row:

In [ ]:
# Model tides and add back into dataframe
df["tide_height"] = model_tides(
    x=df.x,
    y=df.y,
    time=df.time,
    mode="one-to-one",
    directory=directory,
).tide_height.values

# Print dataframe with added tide height data:
df.head()

### "Wide" and "long" output formats
By default, modelled tides will be returned in **"long"** format, with multiple models stacked under a `tide_models` column and tide heights in the `tide_height` column:


In [ ]:
model_tides(
    x=[122.21, 122.22],
    y=[-18.20, -18.21],
    time=pd.date_range(start="2018-01-01", end="2018-01-31", periods=2),
    model=["EOT20", "GOT5.5", "HAMTIDE11"],
    output_format="long",
    directory=directory,
)

We can also run the function in **"wide"** format, which will return a new column for each tide model (e.g. `EOT20`, `GOT5.5`, `HAMTIDE11` etc):

In [ ]:
model_tides(
    x=[122.21, 122.22],
    y=[-18.20, -18.21],
    time=pd.date_range(start="2018-01-01", end="2018-01-31", periods=2),
    model=["EOT20", "GOT5.5", "HAMTIDE11"],
    output_format="wide",
    directory=directory,
)

## Next steps

Now that we have demonstrated how to model tides, we can learn how to [combine modelled tides with satellite data](../Satellite_data) for further analysis.